In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
import os
os.chdir("../..")

In [6]:
import pandas as pd
from pandas_profiling import ProfileReport
import matplotlib.pyplot as plt
import utils.constants as constants

In [7]:
df = pd.read_csv("resources/data/raw/loan.csv")

/tmp/ipykernel_102475/2311671977.py:1: DtypeWarning: Columns (19,55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("resources/data/raw/loan.csv")


In [8]:
colummns_to_drop = (
    constants.CONSTANT_COLUMNS + 
    constants.NULL_COLUMNS + 
    constants.HIGH_CARDINALITY_COLUMNS +
    constants.NOT_ALLOWED_COLUMNS +
    constants.DESCRIPTION_COLUMNS
)
df = df.drop(columns=colummns_to_drop)
df.shape

(887379, 45)

In [9]:
for col in constants.DATE_COLUMNS_TO_SPLIT:
    print(col)
    month_col_name = f"{col}_month"
    year_col_name = f"{col}_year"
    splitted_date_cols = (
        df[col].
        str.
        split("-", expand=True).
        rename(columns={0: month_col_name, 1: year_col_name})
    )
    splitted_date_cols[month_col_name] = splitted_date_cols[month_col_name].map(constants.MONTHS_TO_NUMS_MAPPING)
    df = pd.concat([df, splitted_date_cols], axis=1)
    df = df.drop(columns=[col])

issue_d
earliest_cr_line
last_pymnt_d
last_credit_pull_d


In [7]:
ProfileReport(df, "Initial EDA").to_file("notebooks/eda/profiler.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

Columns to special treatment:
* `mths_since_last_delinq` - categorize + add missing category
* `mths_since_last_record` - categorize + add missing category
* `collections_12_mths_ex_med` - categorize + add missing category
* `issue_d_month` -> sin + cos transformation
* `earliest_cr_line_month` -> sin + cos transformation
* `last_pymnt_d_month` -> sin + cos transformation
* `last_credit_pull_d_month` -> sin + cos transformation
* `issue_d_year` -> scale to $[0, 1]$
* `earliest_cr_line_year` -> scale to $[0, 1]$
* `last_pymnt_d_year` -> scale to $[0, 1]$
* `last_credit_pull_d_year` -> scale to $[0, 1]$

Columns to drop outliers:
* `annual_inc`
* `dti`
* `open_acc`
* `pub_rec`
* `revol_bal`
* `revol_util`
* `total_acc`
* `total_pymnt`
* `total_pymnt_inv`
* `total_rec_prncp`
* `total_rec_late_fee`
* `total_rec_int`
* `recoveries`
* `collection_recovery_fee`
* `last_pymnt_amnt`
* `tot_coll_amt`
* `tot_cur_bal`
* `total_rev_hi_lim`

Columns to reduce skewness:
* `loan_amnt`
* `funded_amnt`
* `funded_amnt_inv`
* `int_rate`
* `annual_inc`
* `revol_bal`
* `total_pymnt`
* `dti`
* `total_pymnt`
* `total_pymnt_inv`
* `total_rec_prncp`
* `total_rec_int`
* `total_rec_late_fee`
* `recoveries`
* `collection_recovery_fee`
* `last_pymnt_amnt`
* `tot_coll_amt`
* `tot_cur_bal`
* `total_rev_hi_lim`

Numerical columns which weren't mentioned:
* `delinq_2yrs`
* `inq_last_6mths`
* `collections_12_mths_ex_med`
* `acc_now_delinq`

In [1]:
from scipy import stats

def get_outliers_thresholds(column, only_positive=False):
    column = column.copy()
    column = column.dropna()
    iqr = stats.iqr(column, nan_policy="raise")
    median = column.median()
    lower_bound = median - 1.5*iqr
    if only_positive:
        lower_bound = max(lower_bound, 0)
    upper_bound = median + 1.5*iqr
    if only_positive:
        upper_bound = max(upper_bound, 0)
    return lower_bound, upper_bound

def cut_outliers(column):
    column = column.copy()
    lower_bound, upper_bound = get_outliers_thresholds(column, only_positive=True)
    column.loc[column < lower_bound] = lower_bound
    column.loc[column > upper_bound] = upper_bound
    return column

In [2]:
cols_to_drop_outliers = [
    "annual_inc",
    "dti",
    "open_acc",
    "pub_rec",
    "revol_bal",
    "revol_util",
    "total_acc",
    "total_pymnt",
    "total_pymnt_inv",
    "total_rec_prncp",
    "total_rec_late_fee",
    "total_rec_int",
    "recoveries",
    "collection_recovery_fee",
    "last_pymnt_amnt",
    "tot_coll_amt",
    "tot_cur_bal",
    "total_rev_hi_lim"
]

In [10]:
df_out = df[cols_to_drop_outliers]

In [13]:
for col in cols_to_drop_outliers:
    df_out.loc[:, col] = cut_outliers(df_out[col])

/tmp/ipykernel_102475/1350105751.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_out.loc[:, col] = cut_outliers(df_out[col])
/tmp/ipykernel_102475/1350105751.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_out.loc[:, col] = cut_outliers(df_out[col])
/tmp/ipykernel_102475/1350105751.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.

In [14]:
df_out.skew()

annual_inc                 0.559431
dti                        0.175679
open_acc                   0.384351
pub_rec                    0.000000
revol_bal                  0.651917
revol_util                -0.171726
total_acc                  0.359886
total_pymnt                0.762504
total_pymnt_inv            0.763321
total_rec_prncp            0.808906
total_rec_late_fee         0.000000
total_rec_int              0.747996
recoveries                 0.000000
collection_recovery_fee    0.000000
last_pymnt_amnt            0.690782
tot_coll_amt               0.000000
tot_cur_bal                0.757966
total_rev_hi_lim           0.652000
dtype: float64